In [ ]:
import os
import tempfile
import requests
import shutil
import re
import gradio as gr

from urllib.request import urlretrieve
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from huggingface_hub import snapshot_download


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

INFO 02-12 04:42:34 [__init__.py:239] Automatically detected platform cuda.


In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

/tmp/ipykernel_1289/983038980.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [ ]:
REPO_ID = "eddyfox8812/otc-chroma-db" 
target_dir = "/workspace/chroma_otc"

local_repo = snapshot_download(repo_id=REPO_ID, repo_type="dataset")

src = os.path.join(local_repo, "chroma_otc")
os.makedirs("/workspace", exist_ok=True)
if os.path.exists(target_dir):
    shutil.rmtree(target_dir)
shutil.copytree(src, target_dir)

print("copied to:", target_dir)
print("files:", os.listdir(target_dir))

vectorstore = Chroma(
    persist_directory=target_dir,
    collection_name="otc_chunks",
    embedding_function=embeddings,
)

print("count:", vectorstore._collection.count())

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

copied to: /workspace/chroma_otc
files: ['4b8e600a-a733-48d2-b496-0752def62f36', 'chroma.sqlite3']


/tmp/ipykernel_1289/3015298251.py:22: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


count: 64015


- drug_names 생성

In [ ]:
col = vectorstore._collection
n = col.count()

drug_set = set()
batch = 5000

for offset in range(0, n, batch):
    out = col.get(include=["metadatas"], limit=batch, offset=offset)
    for m in out["metadatas"]:
        if m and m.get("drug_name"):
            drug_set.add(m["drug_name"])

drug_names = sorted(drug_set)
print("unique drug_names:", len(drug_names))
print("sample:", drug_names[:10])

unique drug_names: 8649
sample: ['1. 포비딘인후스프레이액(포비돈요오드)(바닐라향), 2. 포비딘인후스프레이액(포비돈요오드)(청포도향)', '1.셀레나제100마이크로그램퍼오랄액(아셀렌산나트륨오수화물)2.셀레나제티퍼오랄액(아셀렌산나트륨오수화물)', '1.셀레뉴원오랄액(아셀렌산나트륨오수화물) 2.셀큐민185오랄액(아셀렌산나트륨오수화물)', '1.큐앤큐포비돈거즈볼-중(포비돈요오드)2.큐앤큐포비돈거즈볼-대(포비돈요오드)', '1.포리비돈인후스프레이(포비돈요오드)(바닐라향), 2.포리비돈인후스프레이(포비돈요오드)(청포도향)', '5-엠씨크림(리도카인)', '가그린목액', '가네리버연질캡슐175mg(밀크시슬열매건조엑스)', '가네리버연질캡슐350mg(밀크시슬열매건조엑스)', '가네맥스연질캡슐(밀크시슬열매건조엑스)']


In [ ]:
def candidate_drugs(token: str, drug_names: list[str], top_n: int = 10):
    token = token.strip()
    cands = [d for d in drug_names if token in d]
    cands = sorted(cands, key=len)
    return cands[:top_n]

def choose_best_token(user_query: str, drug_names: list[str], top_n: int = 10):
    tokens = re.findall(r"[가-힣A-Za-z0-9]+", user_query)
    tokens = [t for t in tokens if len(t) >= 2]

    best_token, best_cands = None, []
    for t in tokens:
        cands = candidate_drugs(t, drug_names, top_n=top_n)
        if len(cands) > len(best_cands):
            best_token, best_cands = t, cands

    return best_token, best_cands

def pick_drug_name(user_query: str, top_n=10):
    best_token, cands, tokens = choose_best_token(user_query, top_n=top_n)

    print("\n[token 후보]:", tokens[:15])
    print("[선택된 token]:", best_token, "| 후보 수:", len(cands))

    if not cands:
        print("[drug_name 후보 없음] -> drug_name 필터 없이 진행")
        return None

    print("\n[drug_name 후보 목록]")
    for idx, name in enumerate(cands, 1):
        print(f"{idx}. {name}")

    while True:
        sel = input("번호 선택(Enter=1): ").strip()
        if sel == "":
            sel_idx = 1
            break
        if sel.isdigit():
            sel_idx = int(sel)
            if 1 <= sel_idx <= len(cands):
                break
        print(f"⚠️ 1~{len(cands)} 사이 숫자를 입력하세요.")

    chosen = cands[sel_idx - 1]
    print("선택된 drug_name:", chosen)
    return chosen

def search_from_user_query(user_query: str, k=6, top_n=10, interactive=True):
    chosen_drug_name = pick_drug_name(user_query, drug_names, top_n=top_n, interactive=interactive)

    if chosen_drug_name:
        hits = vectorstore.similarity_search(user_query, k=k, filter={"drug_name": chosen_drug_name})
        print("[필터검색] hits:", len(hits))
        return chosen_drug_name, hits

    hits = vectorstore.similarity_search(user_query, k=k)
    print("[전체검색] hits:", len(hits))
    return None, hits


In [ ]:
_, hits = search_from_user_query("음주 전후에 타이레놀 먹어도돼?", k=6, top_n=10, interactive=True)

[token 후보]: ['음주', '전후에', '타이레놀', '먹어도돼']
[선택된 token]: 타이레놀 | 후보 수: 7
1. 우먼스타이레놀정
2. 타이레놀콜드-에스정
3. 어린이타이레놀현탁액(아세트아미노펜)
4. 타이레놀8시간이알서방정(아세트아미노펜)
5. 타이레놀산500밀리그램(아세트아미노펜)
6. 타이레놀정500밀리그람(아세트아미노펜)
7. 어린이타이레놀산160밀리그램(아세트아미노펜)


번호 선택(Enter=1):  5


선택된 drug_name: 타이레놀산500밀리그램(아세트아미노펜)
[필터검색] hits: 6


In [ ]:
print(f"검색된 문서 수: {len(hits)}")
print(hits[0].metadata)
print(hits[0].page_content[:200])

검색된 문서 수: 6
{'sub_count': 6, 'pro_type': '일반의약품', 'chunk_id': 'a93187414074-warn-1', 'drug_id': 'a93187414074', 'drug_name': '타이레놀산500밀리그램(아세트아미노펜)', 'drug_name_key': '타이레놀산500밀리그램(아세트아미노펜)', 'section': '주의사항', 'sub_index': 1, 'active_ingredient': '[M262287]아세트아미노펜 과립'}
【약물명】타이레놀산500밀리그램(아세트아미노펜)
【주성분】[M262287]아세트아미노펜 과립
【구분】일반의약품
【섹션】주의사항 (1/6)
---
1. 경고
1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열 진통제를 복용해야 할 경우 반드시 의
사 또는 약사와 상의해야 한다. 이러한 사람이 이 약을 복용하면 간손상이 유발될 수 있다.



In [ ]:
vllm_model = LLM(
    model = "eddyfox8812/llama-3-8b-otc-rag-ko-checkpotint-594",
    dtype = "bfloat16"
)

WARNING 02-12 04:43:21 [config.py:2614] Casting torch.float16 to torch.bfloat16.
INFO 02-12 04:43:25 [config.py:585] This model supports multiple tasks: {'classify', 'embed', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 02-12 04:43:25 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.


generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

WARNING 02-12 04:43:27 [utils.py:2181] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/getting_started/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized
INFO 02-12 04:43:30 [__init__.py:239] Automatically detected platform cuda.
INFO 02-12 04:43:31 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='eddyfox8812/llama-3-8b-otc-rag-ko-checkpotint-594', speculative_config=None, tokenizer='eddyfox8812/llama-3-8b-otc-rag-ko-checkpotint-594', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.72it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:02<00:03,  1.62s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:05<00:02,  2.08s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  2.36s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  2.07s/it]



INFO 02-12 04:43:53 [loader.py:447] Loading weights took 8.30 seconds
INFO 02-12 04:43:53 [gpu_model_runner.py:1186] Model loading took 14.9596 GB and 21.143287 seconds
INFO 02-12 04:44:00 [backends.py:415] Using cache directory: /root/.cache/vllm/torch_compile_cache/ecb5e4a8af/rank_0_0 for vLLM's torch.compile
INFO 02-12 04:44:00 [backends.py:425] Dynamo bytecode transform time: 6.96 s
INFO 02-12 04:44:02 [backends.py:132] Cache the graph of shape None for later use
INFO 02-12 04:44:26 [backends.py:144] Compiling a graph for general shape takes 25.28 s
INFO 02-12 04:44:38 [monitor.py:33] torch.compile takes 32.24 s in total
INFO 02-12 04:44:39 [kv_cache_utils.py:566] GPU KV cache size: 397,232 tokens
INFO 02-12 04:44:39 [kv_cache_utils.py:569] Maximum concurrency for 8,192 tokens per request: 48.49x
INFO 02-12 04:44:54 [gpu_model_runner.py:1534] Graph capturing finished in 15 secs, took 0.52 GiB
INFO 02-12 04:44:54 [core.py:151] init engine (profile, create kv cache, warmup model) too

In [ ]:
sampling_params = SamplingParams(
    temperature = 0,
    max_tokens=1024
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("eddyfox8812/llama-3-8b-otc-rag-ko-checkpotint-594")
print(f"모델의 기본 chat template 확인 : {tokenizer.chat_template}")

모델의 기본 chat template 확인 : {% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}


In [ ]:
example_messages = [
    {"role" : "system", "content" : "당신은 도움이 되는 AI 어시스턴트 입니다."},
    {"role" : "user" , "content" : "안녕하세요, 도와주세요" }
]
formatted_example = tokenizer.apply_chat_template(
    example_messages,
    tokenize = False,
    add_generation_prompt = True
)
print("\napply_chat_template 예시 : ")
print(formatted_example)


apply_chat_template 예시 : 
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 도움이 되는 AI 어시스턴트 입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

안녕하세요, 도와주세요<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [ ]:
system_message = """당신은 제공된 [문서]만 근거로 답하는 의약품 RAG 어시스턴트입니다.

필수 규칙:
- 답변에는 반드시 [문서]에서 인용한 근거를 포함해야 합니다.
- 최소 1개 이상, 최대 2개의 '직접 인용'을 포함하세요(문서의 문장을 그대로).
- 직접 인용은 아래 형식으로만 작성:
  근거: [문서N] "인용문"
- 문서에 근거가 없으면 그때만 "문서에 근거가 부족합니다"라고 말하세요.

출력 형식(반드시 지킬 것):
결론: (피해야 함/주의 필요/가능)
이유: (문서 근거 요약 2~3문장)
근거: [문서N] "..."
(필요하면 근거 2개까지)

[문서]
{search_result}
"""

In [ ]:
def format_docs(docs, max_chars_per_doc=1200, include_meta=True):
    if not docs:
        return "관련 문서를 찾지 못했습니다."

    formatted_results = []
    for i, doc in enumerate(docs, 1):
        text = (doc.page_content or "").strip()
        if len(text) > max_chars_per_doc:
            text = text[:max_chars_per_doc] + " ...[truncated]"

        if include_meta:
            m = doc.metadata or {}
            header = f"문서{i} | drug_name={m.get('drug_name')} | section={m.get('section')} | chunk_id={m.get('chunk_id')}"
            formatted_results.append(f"{header}\n{text}")
        else:
            formatted_results.append(f"문서{i} : {text}")

    return "\n----\n".join(formatted_results)


In [ ]:
def candidate_drugs(token: str, top_n=10):
    token = token.strip()
    cands = [d for d in drug_names if token in d]
    cands = sorted(cands, key=len)
    return cands[:top_n]

def choose_best_token(user_query: str, top_n=10):
    tokens = re.findall(r"[가-힣A-Za-z0-9]+", user_query)
    tokens = [t for t in tokens if len(t) >= 2]

    best_token, best_cands = None, []
    for t in tokens:
        cands = candidate_drugs(t, top_n=top_n)
        if len(cands) > len(best_cands):
            best_token, best_cands = t, cands

    return best_token, best_cands, tokens

In [ ]:
def generate_rag_response(question, retriever, tokenizer, vllm_model, sampling_params) : 
    """
    검색 결과와 질문을 기반으로 RAG 응답을 생성하는 함수
    """

    retrieved_docs = retriever.invoke(question)

    formatted_results = format_docs(retrieved_docs)

    formatted_system = system_message.format(search_result = formatted_results)
    messages = [
        {"role" : "system", "content" : formatted_system},
        {"role" : "user", "content" : question}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt = True
    )

    outputs = vllm_model.generate([prompt],sampling_params)

    response = outputs[0].outputs[0].text

    return response, formatted_results, retrieved_docs

In [ ]:
question = "술마시고 타이레놀 먹어도 돼?"
print("질문: ", question)

질문:  술마시고 타이레놀 먹어도 돼?


In [ ]:
chosen_drug_name = pick_drug_name(question, top_n=10)

if chosen_drug_name:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3, "filter": {"drug_name": chosen_drug_name}})
else:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

response, formatted_results, retrieved_docs = generate_rag_response(
    question, retriever, tokenizer, vllm_model, sampling_params
)

print("검색된 문서 수:", len(retrieved_docs))
print("최종답변:\n", response)


[token 후보]: ['술마시고', '타이레놀', '먹어도']
[선택된 token]: 타이레놀 | 후보 수: 7

[drug_name 후보 목록]
1. 우먼스타이레놀정
2. 타이레놀콜드-에스정
3. 어린이타이레놀현탁액(아세트아미노펜)
4. 타이레놀8시간이알서방정(아세트아미노펜)
5. 타이레놀산500밀리그램(아세트아미노펜)
6. 타이레놀정500밀리그람(아세트아미노펜)
7. 어린이타이레놀산160밀리그램(아세트아미노펜)


번호 선택(Enter=1):  5


선택된 drug_name: 타이레놀산500밀리그램(아세트아미노펜)


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.86s/it, est. speed input: 582.42 toks/s, output: 78.61 toks/s]

검색된 문서 수: 3
최종답변:
 결론: 술을 마시고 타이레놀(아세트아미노펜)을 복용하는 것은 주의가 필요합니다.

이유: 매일 세 잔 이상의 술을 마시는 사람이 이 약이나 다른 해열 진통제를 복용해야 할 경우, 반드시 의사 또는 약사와 상의해야 합니다. 이러한 경우에는 간손상이 유발될 수 있습니다. 따라서 술을 마시고 타이레놀을 복용하는 경우에도 이러한 경고 사항을 염두에 두고 복용해야 합니다. [문서1] "매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열 진통제를 복용해야 할 경우 반드시 의사 또는 약사와 상의해야 한다. 이러한 사람이 이 약을 복용하면 간손상이 유발될 수 있다." [문서2] "이 약을 복용하지 말 것"에 포함된 "술을 마시는 사람"에 대한 경고 사항을 참고할 수 있습니다.


In [ ]:
question = "생리통에 탁센 먹어도 돼?"

chosen_drug_name = pick_drug_name(question, top_n=10) 

if chosen_drug_name:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3, "filter": {"drug_name": chosen_drug_name}})
else:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

response, formatted_results, retrieved_docs = generate_rag_response(
    question, retriever, tokenizer, vllm_model, sampling_params
)

print("검색된 문서 수:", len(retrieved_docs))
print("최종답변:\n", response)



[token 후보]: ['생리통에', '탁센', '먹어도']
[선택된 token]: 탁센 | 후보 수: 7

[drug_name 후보 목록]
1. 탁센엠지연질캡슐
2. 탁센이브연질캡슐
3. 탁센레이디연질캡슐
4. 탁센연질캡슐(나프록센)
5. 탁센400이부프로펜연질캡슐
6. 탁센덱시연질캡슐(덱시부프로펜)
7. 탁센아세트아미노펜정500밀리그램(아세트아미노펜)


번호 선택(Enter=1):  5


선택된 drug_name: 탁센400이부프로펜연질캡슐


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.41s/it, est. speed input: 1060.92 toks/s, output: 76.08 toks/s]

검색된 문서 수: 3
최종답변:
 생리통에 탁센400이부프로펜 연질캡슐을 복용해도 되는지에 대한 질문입니다. 이 약물은 일반적으로 생리통 치료에 사용되지 않으며, 복용 시 주의사항이 명시되어 있습니다. 위장관 궤양이나 출혈 환자, 심한 혈액 이상이나 간장애 환자, 심장 기능 부전 환자 등은 이 약을 복용하지 말아야 합니다. 또한, 고혈압 환자나 심혈관계 질환을 가진 환자는 신중히 고려해야 하며, 고용량 사용 시 위험이 증가할 수 있습니다. 따라서 생리통에 탁센을 복용하는 것은 권장되지 않으며, 대신 다른 대체 치료법을 고려하는 것이 좋습니다.


In [ ]:
question = "임산부가 코트리나 먹어도 돼?"

chosen_drug_name = pick_drug_name(question, top_n=10)

if chosen_drug_name:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3, "filter": {"drug_name": chosen_drug_name}})
else:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

response, formatted_results, retrieved_docs = generate_rag_response(
    question, retriever, tokenizer, vllm_model, sampling_params
)

print("검색된 문서 수:", len(retrieved_docs))
print("최종답변:\n", response)


[token 후보]: ['임산부가', '코트리나', '먹어도']
[선택된 token]: 코트리나 | 후보 수: 1

[drug_name 후보 목록]
1. 코트리나캡슐


번호 선택(Enter=1):  1


선택된 drug_name: 코트리나캡슐


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.12s/it, est. speed input: 1402.02 toks/s, output: 75.81 toks/s]

검색된 문서 수: 3
최종답변:
 임산부는 코트리나캡슐을 복용하지 말아야 합니다. 이 약물의 섹션 "주의사항"에 명시된 바와 같이, 임부 및 수유부는 이 약을 복용하지 말아야 한다고 되어 있습니다. 따라서 임산부는 코트리나캡슐을 복용하지 않는 것이 좋습니다.


In [ ]:
question = "유당불내증 있는데 쎄파렉신 먹어도 돼?"

chosen_drug_name = pick_drug_name(question, top_n=10)

if chosen_drug_name:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3, "filter": {"drug_name": chosen_drug_name}})
else:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

response, formatted_results, retrieved_docs = generate_rag_response(
    question, retriever, tokenizer, vllm_model, sampling_params
)

print("검색된 문서 수:", len(retrieved_docs))
print("최종답변:\n", response)


[token 후보]: ['유당불내증', '있는데', '쎄파렉신', '먹어도']
[선택된 token]: 쎄파렉신 | 후보 수: 2

[drug_name 후보 목록]
1. 쎄파렉신캡슐(은교산)
2. 쎄파렉신연조엑스(은교산)


번호 선택(Enter=1):  1


선택된 drug_name: 쎄파렉신캡슐(은교산)


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.01it/s, est. speed input: 3315.07 toks/s, output: 64.41 toks/s]

검색된 문서 수: 3
최종답변:
 문서에 유당불내증에 대한 정보는 제공되지 않았습니다. 따라서 이 질문에 대한 답변을 찾기 어렵습니다.


In [ ]:
def get_candidates(user_query, top_n=10):
    token, cands, _ = choose_best_token(user_query, top_n=top_n)
    return token, cands

def chatbot_respond(message, history, pending):
    if pending is not None:
        sel = message.strip()

        try:
            sel_idx = 1 if sel == "" else int(sel)
        except ValueError:
            return "번호(예: 1,2,3...)로 입력해줘. (Enter면 1번)", pending

        cands = pending["cands"]
        original_q = pending["question"]

        if not cands:
            return "후보가 비어있어. 다시 질문을 입력해줘.", None

        sel_idx = max(1, min(sel_idx, len(cands)))
        chosen = cands[sel_idx - 1]

        retriever = vectorstore.as_retriever(
            search_kwargs={"k": 3, "filter": {"drug_name": chosen}}
        )

        response, _, _ = generate_rag_response(
            question=original_q,
            retriever=retriever,
            tokenizer=tokenizer,
            vllm_model=vllm_model,
            sampling_params=sampling_params,
        )

        return f"선택됨: {chosen}\n\n{response}", None

    token, cands = get_candidates(message, top_n=10)

    if not cands:

        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        response, _, _ = generate_rag_response(
            question=message,
            retriever=retriever,
            tokenizer=tokenizer,
            vllm_model=vllm_model,
            sampling_params=sampling_params,
        )
        return response, None

    menu = "\n".join([f"{i}. {name}" for i, name in enumerate(cands, 1)])
    안내 = (
        "약물명이 여러 개로 검색돼.\n"
        "아래 목록에서 **번호**를 입력해 선택해줘. (Enter면 1번)\n\n"
        f"[token={token} | 후보 {len(cands)}개]\n"
        f"{menu}"
    )

    return 안내, {"question": message, "cands": cands}


with gr.Blocks(theme=gr.themes.Soft()) as demo:
    pending = gr.State(None)

    gr.ChatInterface(
        fn=chatbot_respond,
        additional_inputs=[pending],
        additional_outputs=[pending],
        title="일반의약품 복용방법 및 주의사항 RAG챗봇",
        description="질문을 입력하면 후보 약물명이 나오고, 번호 입력으로 선택할 수 있습니다.",
        examples=[
    ["술마시고 타이레놀 먹어도 돼?", None],
    ["덱시부루펜은 임산부가 먹어도 돼?", None],
    ["7살 아이가 코트리나 먹어도 되는지 알려줘", None],
],
    )

demo.launch(share=True)


/tmp/ipykernel_1289/1812061586.py:72: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://473b3cb0a8f8d2be03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it, est. speed input: 1078.21 toks/s, output: 76.02 toks/s]
